In [42]:
from epub_conversion.utils import open_book, convert_epub_to_lines
import re, time
from tqdm.notebook import tqdm
import pandas as pd
import nltk
import numpy as np

nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from transformers import MarianMTModel, MarianTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/peerchristensen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [43]:
book = open_book("/Users/peerchristensen/Downloads/DaVinciCode.epub")

lines = convert_epub_to_lines(book)

In [44]:
def clean_text(text):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', text)
  return cleantext

cleaned_text = [clean_text(line) for line in lines]

def trim_text(s, n):
    return ' '.join(s.split()[:n])

#trimmed_text = [trim_text(text,500) for text in cleaned_text]

cleaned_text = [text.strip() for text in cleaned_text]

cleaned_text = list(filter(None, cleaned_text))


In [45]:
index = np.arange(len(cleaned_text))

data = pd.DataFrame({'index': index,
                   'text': cleaned_text})

In [46]:
data.head()

,index,text
0,0,The Da Vinci Code
1,1,Dan Brown
2,2,FOR BLYTHE... AGAIN. MORE THAN EVER.
3,3,Acknowledgments
4,4,"First and foremost, to my friend and editor, J..."


In [47]:
def translate(text):
    if text is None or text == "":
        return "Error",

    #batch input + sentence tokenization
    batch = tokenizer.prepare_seq2seq_batch(sent_tokenize(text))

    #run model
    translated = model.generate(**batch)
    tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return " ".join(tgt_text)

In [48]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-da")

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-da")

In [49]:
%%time

data["clean_text"] = data['text'].map(lambda text: clean_text(text))

data['translated'] = data["clean_text"].map(lambda x: translate(x)).copy()

CPU times: user 8h 30min 40s, sys: 34min 36s, total: 9h 5min 16s
Wall time: 4h 42min 11s


In [50]:
data.to_csv('translated.csv')